In [11]:
from pathlib import Path
import numpy as np
path_to_data = Path("/media/hdd1/i-care/10s/0634")

In [12]:
for data in path_to_data.iterdir():
    eeg = np.load(data)
    print(eeg.shape[0])

1000
1000
1000
1000
1000
1000
1000
1000
1000
1002
1000
1002
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1003
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1003
1000
1000
1000
1001
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1001
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1002
1000
1000
1003
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


KeyboardInterrupt: 